In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key3


In [2]:
csv_read = pd.read_csv("../city_data.csv")

In [3]:
city_df = pd.DataFrame(csv_read)
city_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max_temp,Humidity,Clouds,Wind_speed,Country,Date
0,0,kapaa,22.0752,-159.3190,73.0,88,40,9.22,US,1612392712
1,1,yellowknife,62.4560,-114.3525,-15.0,69,90,6.91,CA,1612392485
2,2,clinton,42.5870,-82.9199,37.4,51,20,5.75,US,1612392517
3,3,hobart,-42.8794,147.3294,61.0,94,75,10.36,AU,1612392712
4,4,vaini,-21.2000,-175.2000,82.4,89,75,10.36,TO,1612392712


In [4]:
gmaps.configure(api_key=g_key3)

humidity = city_df["Humidity"].astype(float)
locations = city_df[["Latitude", "Longitude"]]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# perfect weather conditions
perfect_city_df = {}
